In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="HI My name is Fahim, I am noob in AI")])

AIMessage(content="Hello Fahim! \n\nIt's great to meet you. Don't worry about being a noob in AI – everyone starts somewhere! \n\nAI is a fascinating field with a lot to learn, but it's also incredibly rewarding.\n\nWhat are you most interested in learning about AI? \n\nMaybe I can give you some resources or answer some basic questions to get you started. 😄\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 21, 'total_tokens': 105, 'completion_time': 0.152727273, 'prompt_time': 0.000147669, 'queue_time': 0.002949331, 'total_time': 0.152874942}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb9c86a5-3119-4a3e-829c-7aed2051f281-0', usage_metadata={'input_tokens': 21, 'output_tokens': 84, 'total_tokens': 105})

In [4]:
from langchain_core.messages import AIMessage

response = model.invoke([
    HumanMessage(content="HI Im fahim and im noob in AI"),
    AIMessage(content="Hello Fahim! \n\nIt's great to meet you. Don't worry about being a noob in AI, everyone starts somewhere. It's a fascinating field with a lot to learn, but it's also very rewarding"),
    HumanMessage(content="What is my name")
])

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

result = parser.invoke(response)
print(result)

Your name is Fahim, as you told me at the beginning of our conversation. 😊  

Is there anything else I can help you with today, Fahim? 




In [6]:
## Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="HI,I am Fahim and I am a frontend developer")],
    config=config
)

In [11]:
response.content

"Hey Fahim! \n\nWelcome!  It's great to have another frontend developer in the mix.  \n\nWhat are you working on these days? Any exciting projects you're excited about? 😄 💻  \n\n"

In [12]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)

In [13]:
response2.content

'Your name is Fahim. 😄  \n\nI remember!  We met earlier.  What can I help you with today?\n'

In [14]:
#changing the sessionId

config2={"configurable":{"session_id":"chat2"}}
response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config2
)


In [15]:
response2.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [17]:
response3 = with_message_history.invoke(
    [HumanMessage(content="My name is John")],
    config=config2
)

In [18]:
response4 = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config2
)

response4.content

'Your name is John.  \n\nIs there anything else I can help you with? 😊  \n'

Prompt Template

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant answer the questions which are being asked"),
        MessagesPlaceholder(variable_name="messages")

    ]
)

chain = prompt | model

In [20]:
chain.invoke({"messages": [HumanMessage(content="HI My name is fahim")]})

AIMessage(content="Hello Fahim, it's nice to meet you! 👋\n\nHow can I help you today? 😊  Ask me anything! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 27, 'total_tokens': 59, 'completion_time': 0.058181818, 'prompt_time': 0.000166909, 'queue_time': 0.002970761, 'total_time': 0.058348727}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-84ec7ac6-f018-4bbd-832e-85836efc8840-0', usage_metadata={'input_tokens': 27, 'output_tokens': 32, 'total_tokens': 59})

In [21]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [26]:
config3={"configurable":{"session_id":"chat3"}}

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config3
)

In [28]:
response.content

"As a helpful assistant, I don't have access to any personal information about you, including your name.  If you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [51]:
# Addding more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant reply to the question using this {language}"),
        MessagesPlaceholder(variable_name="messages")

    ]
)

chain = prompt | model

In [38]:
response = chain.invoke({"messages":[HumanMessage(content="HI I am Fahim and i dont know what to do in life")],"language":"Tamil"})

In [39]:
response.content

'வணக்கம் Fahim!  \n\nஉன்னைப் பொருத்தவரை வாழ்க்கையில் என்ன செய்ய வேண்டும் என்பது குறித்து தெளிவில்லாதது என்பது புரியலாம்.  கவலைப்படாத, இது ஒரு பொதுவான சந்தேகம். \n\nமேலும் உன்னிடம் சில கேள்விகளுடன் உதவுவேன்:\n\n* **நீ என்னப் படிக்கிறாய்/படித்திருக்கிறாய்?** \n* **உனக்கு எந்த விஷயங்களில் ஆர்வம் இருக்கிறது?** \n* **நீ என்ன பணியில் சேருவது தூண்டுகிறது?** \n* **நீ கவனம் செலுத்தும் போது உனக்கு மகிழ்ச்சியைத் தரு는 விஷயங்கள் என்ன?**\n\nஉனது பதில்களை அடிப்படையாகக் கொண்டு, உங்களுக்காக சில வழிகாட்டிகளைத் தருகிறேன்.  உனது ஆர்வங்களுக்கேற்ப படிப்பு, வேலை, அல்லது பொது நல பணிகளில் ஈடுபடலாம். \n\nவாழ்க்கை ஒரு பயணம், அதில் சரியான வழியைத் தேட வேண்டியது அவசியம்.  உன்னுடைய சக்திகளையும் ஆர்வங்களையும் அறியும்போது,  \nஉன்னைப் பொருத்தமான வாழ்க்கைப் பாதையைத் தீர்மானிக்க முடியும். \n\n'

In [40]:
response = chain.invoke({"messages":[HumanMessage(content="Im preparing for mechanical engineering gate. How to score good marks. give me some strategy")],"language":"Tamil"})
response.content

'##  GATE Mechanical Engineering: Scoring Strategy \n\nவணக்கம்! Mechanical Engineering GATE exam-க்கு தயாராகி இருக்கீங்களே! Good marks score செய்ய  உதவும் சிறந்த  stratgey ங்க  கீழே உள்ளன:\n\n**தொடக்கத்தில்:**\n\n* **Syllabus-ல  சரி  சரி பார்:** GATE syllabus-னை  நன்கு  புரிந்துகொள்ளுங்கள்.  அதில் எந்த அத்தியாயங்கள்  எதுவும் இல்லாத  என்பதை  பாருங்கள்.\n* **Referred Books & Materials:**  நல்ல புத்தகங்கள்  எடுத்துக்கொள்ளுங்கள்.  Previous year papers, sample papers, mock tests  இல்லாமல் இருக்கவே  துல்லியம் இல்லையே! \n* **Time Management:**  ஒவ்வொரு  அத்தியாயத்திற்கும்  சரியான  நேரம்  நீட்டிக்கொள்ளுங்கள். \n\n**படிப்பதற்கு முக்கியம்:**\n\n* **Concept Clarity:** 단순히  தொடர்ந்து  எழுதுவது  மட்டும்  இல்லை,  conceept-ல  நன்கு  புரிந்துகொள்ளுங்கள்.\n* **Practice Problems:**  எல்லா  கொடுக்கப்பட்ட  probelms-லும்  சோதித்துப்  பார்க்கவும். \n* **Revision:**  சிறந்த  revision  மூலம்   எல்லா  அத்தியாயங்கள்  மூலம்  நன்கு  பெறுங்கள்.\n\n**Exam Time:**\n\n* **Question Paper Analysis:**  exam  தொடங்கும்  

In [49]:
## Add more complexity

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)


In [52]:
config4={"configurable":{"session_id":"chat4"}}
response5 = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi  i am fahim")],"language":"hindi"},
    config=config4
)
response5.content

"Hi Fahim!  \n\nI'm glad you said hello!  😊  What's on your mind today? \n\n\n"

Managing the conversation History

In [73]:
from langchain_core.messages import SystemMessage,trim_messages
#trim_messages - helper to reduce how many messages we are sending to the model
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [74]:
messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi I am john"),
    AIMessage(content="Hi John"),
    HumanMessage(content="I like ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 4+5"),
    AIMessage(content="The answer is 10"),
     HumanMessage(content="What is 4-5"),
    AIMessage(content="The answer is -1"),
         HumanMessage(content="What is 4*5"),
    AIMessage(content="The answer is 20"),
]
trimmer.invoke(messages) 
# If you change the token size, it trims the the messages content

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi I am john', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi John', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 4+5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The answer is 10', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 4-5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The answer is -1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 4*5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The answer is 20', additional_kwargs={}, response_metadata={})]

In [75]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    |prompt
    |model
)

response6 = chain.invoke({
    "messages":messages + [HumanMessage(content="What is my name")],
    "language":"English"
}
)

In [76]:
response6.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [77]:
response7 = chain.invoke({
    "messages":messages + [HumanMessage(content="What math problem did I ask")],
    "language":"English"
})

response7.content

'You asked:\n\n* What is 4 + 5 ?\n* What is 4 - 5 ?\n* What is 4 * 5 ? \n\n\n\n\n\n'

In [79]:
# Doing this with message_history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config5={"configurable":{"session_id":"chat5"}}

In [80]:
response7 = with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name")],"language":"English"},
    config=config5
)
response7.content

"As an AI, I have no memory of past conversations and don't know your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"